### Optuna for cnn bi-lstm model

In [2]:
import optuna
import yaml
from src.models import get_model
from src.training.trainer import Trainer
from src.data.dataloader import create_dataloaders  # dataloaders

ModuleNotFoundError: No module named 'src'

In [ ]:


def objective(trial):
    # Load base config
    with open("config/cnn_bilstm.yaml") as f:
        config = yaml.safe_load(f)
    
    # Suggest hyperparameters
    config['training']['optimizer']['learning_rate'] = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    config['model']['lstm_hidden'] = trial.suggest_int("lstm_hidden", 128, 512)
    config['model']['lstm_layers'] = trial.suggest_int("lstm_layers", 1, 4)
    config['training']['gradient_clip'] = trial.suggest_float("grad_clip", 0.5, 2.0)
    
    # Build model
    model = get_model(config['model'])
    
    # Load data
    train_loader, val_loader = create_dataloaders(config['data'])
    
    # Initialize trainer
    trainer = Trainer(model, train_loader, val_loader, config['training'])
    
    # Train for a few epochs to save time
    trainer.train(num_epochs=3)
    
    # Return final validation PER (you want to minimize this)
    return trainer.val_pers[-1]

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # run 20 trials

# Save results
study.trials_dataframe().to_csv("results/optuna_trials.csv", index=False)
print("Best trial:", study.best_trial.params)